# Standard Form conversion

Convert the following linear program into standard form:

$$\begin{aligned}
\text{minimize}\qquad& p + q\\
\text{subject to:}\qquad& 5p - 3q = 7 \\
& 2p + q \ge 2 \\
& 1 \le q \le 4
\end{aligned}$$

### Original problem

In [1]:
using JuMP, Clp

original = Model(solver=ClpSolver())
@variable(original, p)
@variable(original, 1 <= q <= 4 )
@constraint(original, 5p - 3q == 7 )
@constraint(original, 2p + q >= 2 )
@objective(original, Min, p + q )

original

Minimization problem with:
 * 2 linear constraints
 * 2 variables
Solver is Clp

### Standard form
should look like:
$$\begin{aligned}
\text{maximize}\qquad& c^T x\\
\text{subject to:}\qquad& Ax \le b\\
& x \ge 0
\end{aligned}$$

In [9]:
using JuMP, Clp

standard = Model(solver=ClpSolver())
@variable(standard, u >= 0 )
@variable(standard, v >= 0 )
@variable(standard, w >= 0 )
@constraint(standard, (w+1) <= 4 )
@constraint(standard, -5(u-v) + 3(w+1) <= -7 )
@constraint(standard, 5(u-v) - 3(w+1) <= 7 )
@constraint(standard, -2(u-v) - (w+1) <= -2 )
@objective(standard, Max, -(u-v) - (w+1) )

standard

Maximization problem with:
 * 4 linear constraints
 * 3 variables
Solver is Clp

### Standard form (compact)

In [19]:
# we defined the vector x to be: x = [u;  v;  w]
A = [0 0 1; -5 5 3; 5 -5 -3; -2 2 -1]
b = [3; -10; 10; -1]
c = [-1; 1; -1]

using JuMP, Clp

compact = Model(solver=ClpSolver())
@variable(compact, x[1:3] >= 0 )      # specify a vector variable x
@constraint(compact, A*x .<= b )      # the dot in front of <=, 
                                      # which indicates element-wise (vector) inequalities
@objective(compact, Max, c'*x )       # Note the transpose of the cost vector c

compact

Maximization problem with:
 * 4 linear constraints
 * 3 variables
Solver is Clp

### Lets try to solve the problem

The different ways of writing the problem should be \emph{equivalent}. We solve all three to check that we get the same solution.

In [21]:
println("Original problem formulation")

status = solve(original)

println(original)
println(status)
println()
println("p = ", getvalue(p) )
println("q = ", getvalue(q) )
println("objective = ", getobjectivevalue(original) )
println()
println()

println("Problem on standard form")

status = solve(standard)

println(standard)
println(status)
println()
println("p = ", getvalue(u-v) )
println("q = ", getvalue(w+1) )
println("objective = ", -getobjectivevalue(standard) )
println()
println()

println("Problem in compact standard form")

status = solve(compact)

println(compact)
println(status)
println()
println("p = ", getvalue(x[1]-x[2]) )
println("q = ", getvalue(x[3]+1) )
println("objective = ", -(getobjectivevalue(compact)-1) )

Original problem formulation
Min p + q
Subject to
 5 p - 3 q = 7
 2 p + q ≥ 2
 p
 1 ≤ q ≤ 4

Optimal

p = 2.0
q = 1.0
objective = 3.0


Problem on standard form
Max -u + v - w - 1
Subject to
 w ≤ 3
 -5 u + 5 v + 3 w ≤ -10
 5 u - 5 v - 3 w ≤ 10
 -2 u + 2 v - w ≤ -1
 u ≥ 0
 v ≥ 0
 w ≥ 0

Optimal

p = 2.0
q = 1.0
objective = 3.0


Problem in compact standard form
Max -x[1] + x[2] - x[3]
Subject to
 x[3] ≤ 3
 -5 x[1] + 5 x[2] + 3 x[3] ≤ -10
 5 x[1] - 5 x[2] - 3 x[3] ≤ 10
 -2 x[1] + 2 x[2] - x[3] ≤ -1
 x[i] ≥ 0 ∀ i ∈ {1,2,3}

Optimal

p = 2.0
q = 1.0
objective = 3.0
